<a href="https://www.kaggle.com/code/zoendeloi/proyecto-gdd-sofiafraile-fabiangil-enzoloiza?scriptVersionId=141965630" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

[Open in Kaggle](https://www.kaggle.com/code/zoendeloi/proyecto-gdd-sofiafraile-fabiangil-enzoloiza)

# Proyecto Grupal

### Gestión de Datos

Sofía Fraile, Fabián Gil y Enzo Loiza

El siguiente documento corresponde a un informe descriptivo de información rescatada de tres bases de datos: (1) Asistencias a colegios de todo Chile (MINEDUC), (2) ubicación geográfica de los colegios (Propia/Google Maps), y (3) información meteorológica de calidad ambiental (SINCA). Contiene tres partes.

En la primera parte, se realizó un análisis descriptivo de las variables que contienen las bases de datos, y se trabajaron tales variables para que compartan formato entre ellas para simplificar su lectura. En la segunda, se describe el proceso de poblar las bases de datos en SQL. Finalmente, en la tercera parte, se respondieron algunas preguntas relacionadas con las bases de datos.

In [1]:
gc()
rm(list=ls())

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,959928,51.3,1949610,104.2,1249132,66.8
Vcells,1702482,13.0,8388608,64.0,2651364,20.3


In [2]:
# libraries
library("tidyverse")
library("data.table")
library("dplyr")
library("ggplot2")
library("DBI")
library("RPostgreSQL")
library("RODBC")
library("RPostgres")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose




## Documentación de la Base de Datos

Las bases de datos a cargar fueron tres: Asistencias colegios 2018-2019, Calidad de Aire Chile y Localizacion geográfica de colegios, cada una con las variables que se mencionan a continuación:

**Asistencias colegios 2018-2019 (26 variables)**

| N | Nombre | Tipo | Descripción|
|---|-----|------|------------|
| 1 | AGNO | Numeric | Año del proceso |
| 2 | MES_ESCOLAR | Numeric | Mes |
| 3 | RBD | Numeric | Rol Base de Datos del Establecimiento |
| 4 | DGV_RBD | Numeric | Dígito verificador del RBD |
| 5 | NOM_RBD | Character | Nombre del establecimiento |
| 6 | COD_REG_RBD | Numeric | Codigo de region en que se ubica el establecimiento |
| 7 | COD_REG_RBD_A | Character | *Nombre abreviado de la región en que se ubica el establecimiento |
| 8 | COD_PRO_RBD | Numeric | Codigo de la provincia en que se ubica el establecimiento |
| 9 | COD_COM_RBD | Numeric | Código de oficial de comuna en que se ubica el establecimiento |
| 10 | NOM_COM_RBD | Character | Nombre de la comuna |
| 11 | COD_DEPROV_RBD | Numeric | código del departamento provincial en que se ubica el establecimiento |
| 12 | NOM_DEPROV_RBD | Character | Nombre del departamento provinicial donde se ubica el establecimiento | 
| 13 | RURAL_RBD | Numeric | 0 si urbano, 1 si rural |
| 14 | COD_DEPE | Numeric | Dependencia administrativa |
| 15 | COD_DEPE2 | Numeric | Código de dependencia administrativa (agrupado) |
| 16 | COD_ENSE | Numeric | Tipo de enseñanza |
| 17 | COD_ENSE2 | Numeric | Niveles de enseñanza agrupados | 
| 18 | COD_GRADO | Numeric | Código de grado | 
| 19 | LET_CUR | Character | Letra del curso | 
| 20 | MRUN | Numeric | Identificador estudiante máscara | 
| 21 | GEN_ALU | Numeric | Sexo del estudiante |
| 22 | FEC_NAC_ALU | Numeric | Fecha nacimiento alumno AAAAMM (190001/180001 sin info) |
| 23 | COD_COM_ALU | Numeric | Codigo oficial de la comuna de residencia del alumno |
| 24 | NOM_COM_ALU | Character | Nombre comuna de residencia |
| 25 | DIAS_ASISTIDOS | Numeric | Total numero de días asistidos |
| 26 | DIAS_TRABAJADOS | Numeric | Total numero de días trabajados |
| 27 | ASIS_PROMEDIO | Numeric | proporcion de días asistidos respecto de días trabajados |

**Calidad de Aire Chile (12 variables)**

| N | Variable | Tipo | Descripción |
|---|----------|------|-------------|
| 1 | Estacion | Chr | Nombre de la comuna donde está la estación |
| 2 | Fecha (YYMMDD) | int | Fecha de medicion en formato YYMMDD |
| 3 | HORA (HHMM) | int | Hora de medicion en formato HHMM |
| 4 | MP10 | Chr | Material particulado de 10 micrones (decimales con coma) |
| 5 | MP2_5 | Chr | Material part 2.5 micrones (decimales con coma) | 
| 6 | CO | Chr | Monóxido de carbono |
| 7 | NO | Chr | Monoxido nitroso | 
| 8 | NO2 | Chr | Dioxido nitroso |
| 9 | O3 | Chr | Ozono |
| 10 | Humedad | Chr | idem |
| 11 | Temperatura | Chr | idem |
| 12 | Viento_v | Chr | viento |

**Localizacion geográfica de colegios (6 variables)**

| N | Variable | Tipo | Descripción |
|---|----------|------|-------------|
| 1 | RBD | int | Rol de base de datos del colegio |
| 2 | NOM_RBD | chr | Nombre colegio |
| 3 | NOM_COM_RBD | chr | Nombre comuna del establecimiento |
| 4 | query | chr | union de 2 y 3 para busqueda en maps | 
| 5 | lon | chr | longitud geográfica |
| 6 | lat | chr | latitud geográfica |

Sin embargo, las variables no estaban todas en un formato común, por lo que se debió trabajar en ellas para poder consolidarlas.

La primera base de datos (Asistencias colegios 2018-2019) provenía de 20 archivos `csv` donde algunas de éstas tenían 27 variables y otras 26. La variable diferente correspondió a `NOM_REG_RBD_A`, y para efectos de este informe se decidió eliminar la columna. La base se guardó como `db`.

In [3]:
nombre_archivos1 <- c("/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180816_Asistencia_Marzo_2018_20180415_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180830_Asistencia_Abril_2018_20180515_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180720_Asistencia_Mayo_2018_20180615_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180816_Asistencia_Junio_2018_20180715_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20180904_Asistencia_Julio_2018_20180815_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20181026_Asistencia_Agosto_2018_20180915_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20181127_Asistencia_Septiembre_2018_20181015_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20181213_Asistencia_Octubre_2018_20181115_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190107_Asistencia_Noviembre_2018_20181215_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190121_Asistencia_Diciembre_2018_20190115_PUBL.csv")
nombre_archivos2 <- c("/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190514_Asistencia_Marzo_2019_20190415_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20190812_Asistencia_Abril_2019_20190515_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191511_Asistencia_Mayo_2019_20190615_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191211_Asistencia_Junio_2019_20190715_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191211_Asistencia_Julio_2019_20190715_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20191227_Asistencia_Agosto_2019_20190915_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200102_Asistencia_Septiembre_2019_20191015_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200106_Asistencia_Octubre_2019_20191115_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200108_Asistencia_noviembre_2019_20191215_PUBL.csv",
                     "/kaggle/input/asistencia-colegios-2018-y-2019-chile/20200120_Asistencia_diciembre_2019_20200115_PUBL.csv")

db1 <- data.table()

columna_a_eliminar <- "NOM_REG_RBD_A"

In [4]:
for (archivo in nombre_archivos1) {
    data_temp <- fread(archivo, "encoding" ="UTF-8")
    if (columna_a_eliminar %in% colnames(data_temp)){
        data_temp[, (columna_a_eliminar) := NULL]
    }
    db1 <- rbindlist(list(db1, data_temp))
}

In [5]:
for (archivo in nombre_archivos2) {
    data_temp <- fread(archivo, "encoding" ="UTF-8")
    if (columna_a_eliminar %in% colnames(data_temp)){
        data_temp[, (columna_a_eliminar) := NULL]
    }
    db1 <- rbindlist(list(db1, data_temp))
}

In [6]:
str(db1)

Classes ‘data.table’ and 'data.frame':	70143193 obs. of  26 variables:
 $ AGNO           : int  2018 2018 2018 2018 2018 2018 2018 2018 2018 2018 ...
 $ MES_ESCOLAR    : int  3 3 3 3 3 3 3 3 3 3 ...
 $ RBD            : int  1 1 1 1 1 1 1 1 1 1 ...
 $ DGV_RBD        : int  9 9 9 9 9 9 9 9 9 9 ...
 $ NOM_RBD        : chr  "LICEO POLITECNICO ARICA" "LICEO POLITECNICO ARICA" "LICEO POLITECNICO ARICA" "LICEO POLITECNICO ARICA" ...
 $ COD_REG_RBD    : int  15 15 15 15 15 15 15 15 15 15 ...
 $ COD_PRO_RBD    : int  151 151 151 151 151 151 151 151 151 151 ...
 $ COD_COM_RBD    : int  15101 15101 15101 15101 15101 15101 15101 15101 15101 15101 ...
 $ NOM_COM_RBD    : chr  "ARICA" "ARICA" "ARICA" "ARICA" ...
 $ COD_DEPROV_RBD : int  151 151 151 151 151 151 151 151 151 151 ...
 $ NOM_DEPROV_RBD : chr  "ARICA" "ARICA" "ARICA" "ARICA" ...
 $ RURAL_RBD      : int  0 0 0 0 0 0 0 0 0 0 ...
 $ COD_DEPE       : int  2 2 2 2 2 2 2 2 2 2 ...
 $ COD_DEPE2      : int  1 1 1 1 1 1 1 1 1 1 ...
 $ COD_ENSE    

Por el tiempo que toma modificar esta tabla (de alrededor de 7 millones de instancias), se decidió que las variables de las otras bases de datos sean modificadas para parecerse a ésta. La segunda parte de la base de datos se guardó como `db2` y correspondió a los índices de calidad de aire. En esta parte se trabajaron las variables como sigue.

In [7]:
db3 <- fread("/kaggle/input/calidad-del-aire-en-chile/Calidad del aire.csv", "encoding" ="UTF-8")

In [8]:
# ajustes de fechas a formato de db
db3$Fecha <- as.character(db3$"FECHA (YYMMDD)")
db3 <- db3 %>%
    mutate(
        AGNO = as.integer(substr(Fecha, 1, 2)) +
        if_else(as.integer(substr(Fecha, 1, 2)) >= 50, 1900, 2000),
        MES = as.integer(substr(Fecha, 3, 4)),
        DIA = as.integer(substr(Fecha, 5, 6))
    ) %>%
    select(-Fecha) %>%
    select(-"FECHA (YYMMDD)")

# ajustes a MP10, MP2_5, CO, NO, NO2, O3, Humedad, Temperatura, Viento_v
db3 <- db3 %>%
    mutate(
        MP10 = as.numeric(gsub(",", ".", MP10)),
        MP2_5 = as.numeric(gsub(",", ".", MP2_5)),
        CO = as.numeric(gsub(",", ".", CO)),
        NO = as.numeric(gsub(",", ".", NO)),
        NO2 = as.numeric(gsub(",", ".", NO2)),
        O3 = as.numeric(gsub(",", ".", O3)),
        Humedad = as.numeric(gsub(",", ".", Humedad)),
        Temperatura = as.numeric(gsub(",", ".", Temperatura)),
        Viento_v = as.numeric(gsub(",", ".", Viento_v))
    )

db3 <- db3 %>%
    filter(AGNO > 2017) %>%
    filter(AGNO < 2020)

In [9]:
str(db3)

Classes ‘data.table’ and 'data.frame':	3650 obs. of  14 variables:
 $ Estacion   : chr  "Pudahuel" "Pudahuel" "Pudahuel" "Pudahuel" ...
 $ HORA (HHMM): int  0 0 0 0 0 0 0 0 0 0 ...
 $ MP10       : num  41 39 47 37 30 47 61 38 43 55 ...
 $ MP2_5      : num  14 13 11 7 8 12 19 10 11 20 ...
 $ CO         : num  0.18 0.17 0.21 0.13 0.19 0.16 0.37 0.11 0.16 0.18 ...
 $ NO         : num  394 813 879 830 693 ...
 $ NO2        : num  9.5 14.5 15.87 9.52 8.49 ...
 $ O3         : num  23 25 27 19 19 32 35 NA NA NA ...
 $ Humedad    : num  609 630 383 583 767 ...
 $ Temperatura: num  187 183 199 219 200 ...
 $ Viento_v   : num  175.896 212.756 0.543 172.456 159.181 ...
 $ AGNO       : num  2018 2018 2018 2018 2018 ...
 $ MES        : int  1 1 1 1 1 1 1 1 1 1 ...
 $ DIA        : int  1 2 3 4 5 6 7 8 9 10 ...
 - attr(*, ".internal.selfref")=<externalptr> 


Además, incluimos en esta parte las coordenadas de las estaciones de monitoreo.

In [10]:
unique(db3$Estacion) #'Pudahuel''Las Condes''Santiago''El Bosque''La Florida'

db3 <- db3 %>%
    mutate(lat = case_when(Estacion == "Las Condes" ~ -33.460277,
                          Estacion == "Santiago" ~ -33.460278,
                          Estacion == "Pudahuel" ~ -33.460325,
                          Estacion == "El Bosque" ~ -33.460325,
                          Estacion == "La Florida" ~ -33.460277)
          )
db3 <- db3 %>%
    mutate(lon = case_when(Estacion == "Las Condes" ~ -70.65861,
                          Estacion == "Santiago" ~ -70.65833,
                          Estacion == "Pudahuel" ~ -70.65833,
                          Estacion == "El Bosque" ~ -70.65833,
                          Estacion == "La Florida" ~ -70.65833)
          )

[1] "Pudahuel"   "Las Condes" "Santiago"   "El Bosque"  "La Florida"

In [11]:
db4 <- fread("/kaggle/input/localizacion-geografica-de-colegios-de-chile/colesgeo.csv", "encoding" = "UTF-8")

## Poblar Bases de Datos



Entidades

In [12]:
# estudiantes
estudiantes <- db1[, c("MRUN", "MES_ESCOLAR", "ASIS_PROMEDIO")] %>%
    group_by(MRUN) %>%
    filter(n() == 20) %>%
    ungroup()

In [13]:
write.csv("estudiantes.csv", row.names = TRUE)

"","x"
"1","estudiantes.csv"


In [14]:
 # comprobar

In [15]:
estudiantes <- estudiantes[ !duplicated(estudiantes$MRUN), ]

In [16]:
# colegios

In [17]:
# asistencia

In [18]:
system("sudo apt install -y postgresql postgresql-contrib &>log", intern=T)
system("service postgresql start", intern = T)
system("sudo -u postgres psql -c \"CREATE USER root WITH SUPERUSER\"", intern=T)
system("sudo -u postgres createdb mineduc", intern=T)

[1] "Reading package lists..."                                                                                           
 [2] "Building dependency tree..."                                                                                        
 [3] "Reading state information..."                                                                                       
 [4] "The following additional packages will be installed:"                                                               
 [5] "  cron libpopt0 libxslt1.1 logrotate postgresql-12 postgresql-client-12"                                            
 [6] "  postgresql-client-common postgresql-common ssl-cert sysstat"                                                      
 [7] "Suggested packages:"                                                                                                
 [8] "  anacron checksecurity default-mta | mail-transport-agent bsd-mailx | mailx"                                       
 [9] "  postgresql-doc postgresql-doc-12 libjson-perl openssl-blacklist isag"                                             
[10] "The following NEW packages will be installed:"                                                                      
[11] "  cron libpopt0 libxslt1.1 logrotate postgresql postgresql-12"                                                      
[12] "  postgresql-client-12 postgresql-client-common postgresql-common"                                                  
[13] "  postgresql-contrib ssl-cert sysstat"                                                                              
[14] "0 upgraded, 12 newly installed, 0 to remove and 8 not upgraded."                                                    
[15] "Need to get 15.5 MB of archives."                                                                                   
[16] "After this operation, 48.5 MB of additional disk space will be used."                                               
[17] "Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 cron amd64 3.0pl1-136ubuntu1 [71.5 kB]"                     
[18] "Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libpopt0 amd64 1.16-14 [26.3 kB]"                           
[19] "Get:3 http://archive.ubuntu.com/ubuntu focal/main amd64 logrotate amd64 3.14.0-4ubuntu3 [44.5 kB]"                  
[20] "Get:4 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libxslt1.1 amd64 1.1.34-4ubuntu0.20.04.1 [151 kB]"  
[21] "Get:5 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 postgresql-client-common all 214ubuntu0.1 [28.2 kB]"
[22] "Ign:6 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 postgresql-client-12 amd64 12.15-0ubuntu0.20.04.1"  
[23] "Get:7 http://archive.ubuntu.com/ubuntu focal/main amd64 ssl-cert all 1.0.39 [17.0 kB]"                              
[24] "Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 postgresql-common all 214ubuntu0.1 [169 kB]"        
[25] "Ign:9 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 postgresql-12 amd64 12.15-0ubuntu0.20.04.1"         
[26] "Get:10 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 postgresql all 12+214ubuntu0.1 [3,924 B]"          
[27] "Get:11 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 postgresql-contrib all 12+214ubuntu0.1 [3,932 B]"  
[28] "Get:12 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 sysstat amd64 12.2.0-2ubuntu0.3 [448 kB]"          
[29] "Err:6 http://security.ubuntu.com/ubuntu focal-updates/main amd64 postgresql-client-12 amd64 12.15-0ubuntu0.20.04.1" 
[30] "  404  Not Found [IP: 91.189.91.81 80]"                                                                             
[31] "Err:9 http://security.ubuntu.com/ubuntu focal-updates/main amd64 postgresql-12 amd64 12.15-0ubuntu0.20.04.1"        
[32] "  404  Not Found [IP: 91.189.91.81 80]"                                                                             
[33] "Fetched 964 kB in 1s (851 kB/s)"

Warning message in system("service postgresql start", intern = T):
“running command 'service postgresql start' had status 1”


character(0)
attr(,"status")
[1] 1
attr(,"errmsg")
[1] "Resource temporarily unavailable"

Warning message in system("sudo -u postgres psql -c \"CREATE USER root WITH SUPERUSER\"", :
“running command 'sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"' had status 1”


character(0)
attr(,"status")
[1] 1
attr(,"errmsg")
[1] "Resource temporarily unavailable"

Warning message in system("sudo -u postgres createdb mineduc", intern = T):
“running command 'sudo -u postgres createdb mineduc' had status 1”


character(0)
attr(,"status")
[1] 1
attr(,"errmsg")
[1] "Resource temporarily unavailable"

## Preguntas Específicas

Lorem ipsum.